In [1]:
!pip install pypdf
!pip install pymorphy2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 990.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 18.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=fa9d1115b0b31f4ac31792b10e9dba955c4f18ef01c948cb9829dac35f50641d
  Stored in directory: /root/.cache/pip/wheels/fc/ab/d4/5da2067ac95b36618c629a5f93f809425700506f72c9732fac
Successfully built docopt


In [2]:
import os
import warnings
import requests
import urllib3
from urllib.request import urlopen
from urllib.parse import urlparse
from bs4 import BeautifulSoup
from pypdf import PdfReader
import pandas as pd
import numpy as np
import re
import nltk
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from pymorphy2 import MorphAnalyzer

warnings.filterwarnings("ignore")
urllib3.disable_warnings()

In [3]:
from bs4 import BeautifulSoup


sample_html_code = '''
<html>
  <body>
    <h1>This is a heading</h1>
    <p>This is a paragraph.</p>
    <p>Another paragraph.</p>
  </body>
</html>
'''

def MarkText(html_code : str):

    soup = BeautifulSoup(html_code, 'html.parser')

    headings = ['h1', 'h2', 'h3', 'h4', 'h5', 'h6']
    paragraphs = ['p', 'pre']

    HEADING_MARK = '//@'
    PARAGRAPH_MARK = '//&'

    def process_tag(tag):
        if tag.name in headings:
            return HEADING_MARK + tag.text.strip() + '\n'
        elif tag.name in paragraphs:
            return PARAGRAPH_MARK + tag.text.strip() + '\n'
        else:
            return ''


    result = ''

    for tag in soup.find_all():
        result += process_tag(tag)

    return result
print(MarkText(sample_html_code))

//@This is a heading
//&This is a paragraph.
//&Another paragraph.



### Ссылки на ESG рейтинг RAEX

* https://www.raexpert.eu/esg_corporate_ranking/#conf-tab-1
* https://raex-rr.com/ESG/ESG_companies/ESG_rating_companies/2023.4/

## Сбор данных RAEX

### Парсинг сайтов компаний из рейтинга

In [5]:
nltk.download("stopwords")
stopwords_ru = stopwords.words("russian")
morph = MorphAnalyzer()

headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/83.0.4103.106 Safari/537.36',
    "accept": "application/json"
}

timeout = (5, 25)
RAEX_PATH = '/content/RAEX list.xlsx'
raex_list = pd.read_excel("/content/RAEX list.xlsx").dropna(subset=["url_sustainability"]).reset_index(drop=True)
raex_list

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,№,Название,Код MOEX,Подотрасль,ESG-рейтинг,E Rank,E-рейтинг,S Rank,S-рейтинг,G Rank,G-рейтинг,Год последней оцененной отчетности,url,url_sustainability
0,1,НЛМК,NLMK,Чёрная металлургия,AA,2,AA,2,AA,21,A,2021,https://nlmk.com/ru/,https://nlmk.com/ru/sustainability/
1,2,«Полюс»,PLZL,Драгоценные металлы,AA,1,AAA,14,A,27,A,2021,https://polyus.com/ru/,https://sustainability.polyus.com/ru/
2,3,«Уралкалий»,-,Агрохимикаты,A,6,BBB,1,AA,6,AA,2021,https://www.uralkali.com/ru/,https://www.uralkali.com/ru/sustainability/
3,4,«ЭЛ5-Энерго»,ELFV,Электроэнергетика,A,9,BBB,9,A,2,AAA,2021,https://www.el5-energo.ru,https://www.el5-energo.ru/sustainability/
4,5,«Полиметалл»,POLY,Драгоценные металлы,A,4,A,5,A,12,AA,2021,https://www.polymetalinternational.com/ru/,https://www.polymetalinternational.com/ru/sust...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
136,153,Кордиант,-,Производство шин,C,154,C,154,C,142,CC,2020,https://cordiant.ru,https://www.cordiant-tyre.ru/values/
137,155,«Промышленно-металлургический холдинг» (ПМХ),-,Чёрная металлургия,C,156,C,148,CC,149,CC,2020,https://www.metholding.ru,https://www.metholding.ru/development/
138,156,"""Титан"", группа компаний (деревообработка)",-,Деревообработка,C,148,C,152,C,152-154,CC,2020,https://titan-group.ru,https://titan-group.ru/about/development/
139,157,«Минудобрения»,-,Агрохимикаты,C,157,C,157,C,157-160,C,2021,https://minudo.ru,https://minudo.ru/?cid=28&parent_id=5


In [6]:
def parsing_pdf(company, pdf):
    reader = PdfReader(f"pdf/{pdf}")
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"

    text = text.replace('-\n', '').lower()
    text = re.sub(r'[`!@#$%^&*()_+\-=\[\]{};\':"\\|,.<>\/?~©«»—]', r'', text)
    text = ''.join([i for i in text if not i.isdigit()])

    lines = (line.strip() for line in text.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    text = '\n'.join(chunk for chunk in chunks if chunk)

    text = list(np.concatenate([sent_tokenize(i.strip()) for i in text.split('\n')]).flat)

    text_lemmatized = []
    for line in [t.split() for t in text]:
        line_lemmatized = ' '.join([morph.normal_forms(l)[0] for l in line if l not in stopwords_ru])
        text_lemmatized.append(line_lemmatized)

    data = pd.DataFrame(text_lemmatized, columns=['text']).drop_duplicates()
    data['company'] = 'Заглушка'
    data['rating'] = 0
    data['url'] = pdf
    data = data[['rating', 'company', 'url', 'text']]

    return data

In [7]:
#сюда вставить sql или чето типа
import sqlite3

conn = sqlite3.connect('history.db')
cur = conn.cursor()

cur.execute('''CREATE TABLE Pages (
    PageID INT PRIMARY KEY,
    ParentPageID INT,
    PageLevel INT,
    PageURL TEXT,
    PageText TEXT)
''')



In [ ]:
def parsing_site(company, url, depth, max_depth=0):
#     print(url)
    internalLinks = [(url, '/', 0)] #  Создаем массив ссылок, которые обрабатываем обходом в ширину.
    used = []
    path_from = {}
    megadata = pd.DataFrame()
    while internalLinks:
        #  new
        now = internalLinks.pop(0)
        url = now[0]
        from_url = now[1]
        depth = now[2]
        #  Помечаем ссылку как использованную, чтоб не зациклиться.
        used.append(url)
        # Сохраняем ссылку откуда пришли, чтобы потом восстановить путь до корня.
        path_from[url] = from_url
        #  new

        response = requests.get(url=url, timeout=timeout, headers=headers, verify=False)
        soup = BeautifulSoup(response.content, 'lxml')

        for script in soup(["script", "style"]):
            script.extract()

        text = soup.get_text()

        lines = (line.strip() for line in text.splitlines())
        chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
        text = '\n'.join(chunk for chunk in chunks if chunk).replace('-\n', '').lower()
        text = re.sub(r'[`!@#$%^&*()_+\-=\[\]{};\':"\\|,.<>\/?~©«»—]', r'', text)
        text = ''.join([i for i in text if not i.isdigit()])

        text = list(np.concatenate([sent_tokenize(i.strip()) for i in text.split('\n')]).flat)

        text_lemmatized = []
        for line in [t.split() for t in text]:
            line_lemmatized = ' '.join([morph.normal_forms(l)[0] for l in line if l not in stopwords_ru])
            text_lemmatized.append(line_lemmatized)

        spans = list(filter(None, [span.string for span in soup.find_all('span')]))

        if len(spans):
            spans = '\n'.join(span for span in spans if span).replace('-\n', '').lower()
            spans = re.sub(r'[`!@#$%^&*()_+\-=\[\]{};\':"\\|,.<>\/?~©«»—]', r'', spans)
            spans = ''.join([i for i in spans if not i.isdigit()])

            spans = list(np.concatenate([sent_tokenize(i.strip()) for i in spans.split('\n')]).flat)
        else:
            spans = []

        spans_lemmatized = []
        for line in [s.split() for s in spans]:
            line_lemmatized = ' '.join([morph.normal_forms(l)[0] for l in line if l not in stopwords_ru])
            spans_lemmatized.append(line_lemmatized)

        df1 = pd.DataFrame(text_lemmatized, columns=['text'])
        df2 = pd.DataFrame(spans_lemmatized, columns=['text'])
        data = pd.concat([df1, df2]).drop_duplicates().reset_index(drop=True)
        data['company'] = company['Название']
        data['rating'] = company['№']
        data['url'] = url
        data = data[['rating', 'company', 'url', 'text']]
        # По идее, в конце работы программы, в megadata должно храниться ваще все.
        megadata = pd.concat([megadata, data]).drop_duplicates()
        # Считываем все ссылки на сайте и добавляем их в конец нашей очереди обхода в ширину.
        # Проверяем так же глубину обхода - слишком далеко нам уходить не надо.
        if depth < max_depth:
            NewLinks = [
                a.get('href') for a in soup.find_all('a')
                if a.get('href') and a.get('href').startswith('/')
            ]
            NewLinks = [link[1:] for link in NewLinks]
            NewLinks = [*set(NewLinks)]
            NewLinks = list(filter(None, NewLinks))

            for link in NewLinks:
                    if ".pdf" not in link and ".PDF" not in link:
                        new_url = os.path.join(url[:url.rfind('www')], urlparse(url).netloc, link)
                        #Тут как раз добавляем URL в конец очереди.
                        if new_url not in used:
                            internalLinks.append((new_url, url, depth + 1))

        #  Здесь вставляем наш PAGE_ID
        #  Путь до корневой папки и будет иерархической определяющей нашей страницы.
        #  Восстанавливается он следующим образом :
        path_now = []
        while url != '/' :
            path_now.append(url)
            url = path_from[url]
        path_now.reverse()

        # В path_now хранится путь до страницы из корня!
    return megadata

In [22]:
CROP_TO_SHOW = 50
from urllib.parse import urljoin

def parsing_site_demo(company, url, depth, max_depth=0):
#     print(url)
    internalLinks = [(url, '/', 0)] #  Создаем массив ссылок, которые обрабатываем обходом в ширину.
    used = []
    path_from = {}
    megadata = pd.DataFrame()
    while internalLinks:
        #  new
        now = internalLinks.pop(0)
        url = now[0]
        from_url = now[1]
        depth = now[2]
        #  Помечаем ссылку как использованную, чтоб не зациклиться.
        used.append(url)
        # Сохраняем ссылку откуда пришли, чтобы потом восстановить путь до корня.
        path_from[url] = from_url
        #  new

        response = requests.get(url=url, timeout=timeout, headers=headers, verify=False)
        text = MarkText(response.text)[:CROP_TO_SHOW]
        data = pd.DataFrame()
        data['company'] = [company['Название']]
        data['rating'] = [company['№']]
        data['url'] = [url]
        data['text'] = [text]
        #  print(text)
        soup = BeautifulSoup(response.content, 'lxml')

        # data = data[['rating', 'company', 'url', 'text']]
        # По идее, в конце работы программы, в megadata должно храниться ваще все.

        # Считываем все ссылки на сайте и добавляем их в конец нашей очереди обхода в ширину.
        # Проверяем так же глубину обхода - слишком далеко нам уходить не надо.
        if depth < max_depth:
            NewLinks = [
                a.get('href') for a in soup.find_all('a')
                if a.get('href') and a.get('href').startswith('/')
            ]
            print(NewLinks)
            NewLinks = [link[1:] for link in NewLinks]
            NewLinks = [*set(NewLinks)]
            NewLinks = list(filter(None, NewLinks))

            for link in NewLinks:
                    if ".pdf" not in link and ".PDF" not in link:

                        #  new_url = os.path.join(url[:url.rfind('www')], urlparse(url).netloc, link)
                        new_url = urljoin(url[:url.rfind('.com')] + '.com', link)
                        print(new_url)
                        #Тут как раз добавляем URL в конец очереди.
                        if new_url not in used:
                            internalLinks.append((new_url, url, depth + 1))

        #  Путь до корневой папки и будет иерархической определяющей нашей страницы.
        #  Восстанавливается он следующим образом :
        path_now = []
        print(url, 'downloaded successfully!')
        while url != '/' :
            path_now.append(url)
            url = path_from[url]
        path_now.reverse()
        data['path'] = path_now
        data['depth'] = depth
        #  print(data['text'])
        megadata = pd.concat([megadata, data])

        # В path_now хранится путь до страницы из корня!
    return megadata

Здесь протестируем работу алгоритма на 10 предложенных сайтах)

In [24]:
nltk.download('punkt')
max_depth = 0
big_df = pd.DataFrame()
for i, company in raex_list.iterrows():
    rating = company['№']
    name = company['Название']
    url = company['url_sustainability']
    df = parsing_site_demo(company, url, 0, max_depth)
    big_df = pd.concat([big_df, df])
    if (i == 9) :
        print('Демо-версия окончена! Дальше надо платить деньги.')
        break
big_df

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


https://nlmk.com/ru/sustainability/ downloaded successfully!
https://sustainability.polyus.com/ru/ downloaded successfully!
https://www.uralkali.com/ru/sustainability/ downloaded successfully!
https://www.el5-energo.ru/sustainability/ downloaded successfully!
https://www.polymetalinternational.com/ru/sustainability/ downloaded successfully!
https://www.phosagro.ru/sustainability/ downloaded successfully!
https://ir.mkb.ru/sustainability downloaded successfully!
https://www.nornickel.ru/sustainability/ downloaded successfully!
https://severstal.com/rus/sustainable-development/ downloaded successfully!
https://lukoil.ru/Sustainability downloaded successfully!
Демо-версия окончена! Дальше надо платить деньги.


,company,rating,url,text,path,depth
0,НЛМК,1,https://nlmk.com/ru/sustainability/,//@Устойчивое развитие\n//@Наш подход\n//&Явля...,https://nlmk.com/ru/sustainability/,0
0,«Полюс»,2,https://sustainability.polyus.com/ru/,//&RAEX подтвердило ESG-рейтинг «Полюса» на ур...,https://sustainability.polyus.com/ru/,0
0,«Уралкалий»,3,https://www.uralkali.com/ru/sustainability/,,https://www.uralkali.com/ru/sustainability/,0
0,«ЭЛ5-Энерго»,4,https://www.el5-energo.ru/sustainability/,//@Активы\n//@Руководство\n//@Карьера\n//@Гене...,https://www.el5-energo.ru/sustainability/,0
0,«Полиметалл»,5,https://www.polymetalinternational.com/ru/sust...,//@Устойчивое развитие\n//&В свете произошедши...,https://www.polymetalinternational.com/ru/sust...,0
0,"«ФосАгро», группа",6,https://www.phosagro.ru/sustainability/,//@Устойчивое развитие\n//@Управление устойчив...,https://www.phosagro.ru/sustainability/,0
0,«МОСКОВСКИЙ КРЕДИТНЫЙ БАНК»,7,https://ir.mkb.ru/sustainability,//@С заботой о будущем\n,https://ir.mkb.ru/sustainability,0
0,"«Норильский никель», горно-металлургическая ко...",8,https://www.nornickel.ru/sustainability/,,https://www.nornickel.ru/sustainability/,0
0,«Северсталь»,9,https://severstal.com/rus/sustainable-developm...,//@Устойчивое\r\nразвитие\n//@Отчет об устойчи...,https://severstal.com/rus/sustainable-developm...,0
0,«ЛУКОЙЛ»,10,https://lukoil.ru/Sustainability,//@block\n//@Web Page Blocked!\n//&The page ca...,https://lukoil.ru/Sustainability,0


In [ ]:
%%time
max_depth = 1
parsed_data = pd.DataFrame()

# raex_list = ['http://www.sberbank.ru/ru/person']

# for i, company in raex_list[140:141].iterrows():
# for i, company in raex_list[7:8].iterrows():
for i, company in raex_list.iterrows():
    rating = company['№']
    name = company['Название']
    url = company['url_sustainability']

    try:
        if ".pdf" in url:
            df = parsing_pdf(company, url)
        else:
            df = parsing_site(company, url, 0, max_depth)
        parsed_data = pd.concat([parsed_data, df])
        print('DONE', i, name)
    except Exception as err:
        print('ERROR', i, name, err)
        continue

parsed_data = parsed_data.drop_duplicates()
parsed_data = parsed_data.reset_index(drop=True)
parsed_data.to_csv('parsed_data.csv', encoding='utf-8')

In [ ]:
parsed_data

In [ ]:
parsed_data.to_csv('parsed_data.csv', encoding='utf-8')

### Тесты и проверки

In [ ]:
company = raex_list.loc[7]
# company = raex_list.loc[140]
# company = raex_list.loc[12]
# company = raex_list.loc[18]
# company = raex_list.loc[31]
# company = raex_list.loc[2]
company = raex_list.loc[27]
name = company['Название']
url = company['url_sustainability']
company

In [ ]:
response = requests.get(url=url, timeout=timeout, headers=headers, verify=False)
soup = BeautifulSoup(response.content, 'lxml')

for script in soup(["script", "style"]):
    script.extract()

internalLinks = [
    a.get('href') for a in soup.find_all('a')
    if a.get('href') and a.get('href').startswith('/')
]
internalLinks = [link[1:] for link in internalLinks]
internalLinks = [*set(internalLinks)]
internalLinks = list(filter(None, internalLinks))

print(url)

internalLinks

In [ ]:
for link in internalLinks:
    if ".pdf" not in link:
        new_url = os.path.join(url[:url.rfind('www')], urlparse(url).netloc, link)
        print(new_url)

In [ ]:
max_depth = 1
parsing_site(company, url, 0, max_depth)

In [ ]:
parsing_pdf(company, url)